In [2]:
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
  Using cached joblib-1.4.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached threadpoolctl-3.5.0-py3-none-any.whl.metadata (13 kB)
   ---------------------------------------- 0.0/11.1 MB ? eta -:--:--
   --- ------------------------------------ 1.0/11.1 MB 6.3 MB/s eta 0:00:02
   --------- ------------------------------ 2.6/11.1 MB 6.0 MB/s eta 0:00:02
   -------------- ------------------------- 3.9/11.1 MB 6.3 MB/s eta 0:00:02
   ------------------ --------------------- 5.2/11.1 MB 6.4 MB/s eta 0:00:01
   ----------------------- ---------------- 6.6/11.1 MB 6.4 MB/s eta 0:00:01
   ----------------------------- ---------- 8.1/11.1 MB 6.4 MB/s eta 0:00:01
   --------------------------------- ------ 9.4/11.1 MB 6.4 MB/s eta 0:00:01
   -------------------------------------- - 10.7/11.1 MB 6.3 MB/s eta 0:00:01
   ---------------------------------------- 11.1/11.1 MB 6.1 MB/s eta 0:00:00
Using cached joblib-1.4.2


[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd

# Load your dataset
df = pd.read_csv("rximage_dataV3.csv")

# Drop duplicates based on base_filename to get unique pills
unique_pills = df.drop_duplicates(subset=["base_filename"])

# Randomly sample 500 unique pills
subset_unique_pills = unique_pills.sample(n=500, random_state=42)

# Get all augmented images for the selected pills
subset_df = df[df["base_filename"].isin(subset_unique_pills["base_filename"])]

# Save the subset to a new CSV
subset_df.to_csv("pill_subset500.csv", index=False)

print("Subset of 500 pills (with all augmented versions) saved to pill_subset.csv")

Subset of 500 pills (with all augmented versions) saved to pill_subset.csv


In [2]:
import os
import shutil
import random
import pandas as pd
import glob

# Define paths
dataset_path = "D:/rximage/image/images/split_padded_rotated"  # Folder where all images are stored
output_dir = "dataset"  # Output directory for train/test split
train_dir = os.path.join(output_dir, "train")
test_dir = os.path.join(output_dir, "test")

# Load dataset
df = pd.read_csv("pill_subset500.csv")
base_filenames = df["base_filename"].unique()

# Ensure output directories exist
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Function to copy images into train and test directories
def copy_images(pill_set, target_dir):
    for base_filename in pill_set:
        # Get all images for this base_filename
        filename_prefix = df[df["base_filename"] == base_filename]["new_filename"].iloc[0]  # Example file
        filename_prefix = os.path.splitext(filename_prefix)[0]  # Remove extension
        matching_images = glob.glob(os.path.join(dataset_path, f"{filename_prefix}*.*"))  # Match all variations

        # Split the images into training and testing (80-20 split)
        random.shuffle(matching_images)  # Shuffle to ensure randomness
        split_idx = int(0.8 * len(matching_images))  # 80% for training, 20% for testing
        train_images = matching_images[:split_idx]
        test_images = matching_images[split_idx:]

        # Create class folder
        class_train_dir = os.path.join(target_dir, "train", str(base_filename))
        class_test_dir = os.path.join(target_dir, "test", str(base_filename))
        os.makedirs(class_train_dir, exist_ok=True)
        os.makedirs(class_test_dir, exist_ok=True)

        # Copy images into train/test directories
        for img_path in train_images:
            if os.path.exists(img_path):
                shutil.copy(img_path, os.path.join(class_train_dir, os.path.basename(img_path)))
            else:
                print(f"Warning: {img_path} not found!")

        for img_path in test_images:
            if os.path.exists(img_path):
                shutil.copy(img_path, os.path.join(class_test_dir, os.path.basename(img_path)))
            else:
                print(f"Warning: {img_path} not found!")

# Copy images into train and test sets
copy_images(base_filenames, output_dir)

print("Dataset successfully split into training and testing sets with shared pills!")

Dataset successfully split into training and testing sets with shared pills!


In [ ]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision import datasets, models
from tqdm import tqdm

# Paths to dataset
train_dir = "dataset/train"
test_dir = "dataset/test"

# Hyperparameters
batch_size = 32
num_epochs = 10
learning_rate = 0.001
num_classes = 500  # Since you have 500 different pills

# Transformations for data augmentation and normalization
transform = transforms.Compose([
    transforms.Resize((224, 224)),  # Resize images to 224x224 (ResNet input size)
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),  # Pre-trained ResNet stats
])

# Load the datasets
train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
test_dataset = datasets.ImageFolder(root=test_dir, transform=transform)

# Create data loaders for training and testing
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Load the pre-trained ResNet model
model = models.resnet18(pretrained=True)  # You can also use resnet34, resnet50, etc.
model.fc = nn.Linear(model.fc.in_features, num_classes)  # Modify the final layer to match the number of classes (50)

# Move the model to the GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()  # Cross-entropy loss for classification
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

# Training loop
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0
    for inputs, labels in tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs}"):
        inputs, labels = inputs.to(device), labels.to(device)
        
        # Zero the parameter gradients
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        
        # Backward pass and optimize
        loss.backward()
        optimizer.step()

        # Statistics
        running_loss += loss.item()
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

    # Calculate training accuracy
    train_accuracy = 100 * correct / total
    print(f"Epoch {epoch+1}/{num_epochs} - Loss: {running_loss/len(train_loader):.4f} - Accuracy: {train_accuracy:.2f}%")

    # Optionally, save the model after every epoch
    torch.save(model.state_dict(), f"model_epoch_{epoch+1}.pth")

# Testing loop
model.eval()  # Set the model to evaluation mode
correct = 0
total = 0
with torch.no_grad():
    for inputs, labels in tqdm(test_loader, desc="Testing"):
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

# Calculate test accuracy
test_accuracy = 100 * correct / total
print(f"Test Accuracy: {test_accuracy:.2f}%")

C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\dougl\AppData\Roaming\Python\Python312\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Epoch 1/10: 100%|██████████| 375/375 [2:14:39<00:00, 21.55s/it]  


Epoch 1/10 - Loss: 4.9014 - Accuracy: 6.43%


Epoch 2/10: 100%|██████████| 375/375 [2:05:27<00:00, 20.07s/it]  


Epoch 2/10 - Loss: 2.0284 - Accuracy: 41.82%


Epoch 3/10: 100%|██████████| 375/375 [2:22:02<00:00, 22.73s/it]  


...

Epoch 6/10: 100%|██████████| 375/375 [1:58:12<00:00, 18.91s/it]


Epoch 6/10 - Loss: 0.2801 - Accuracy: 91.12%


Epoch 7/10: 100%|██████████| 375/375 [1:25:47<00:00, 13.73s/it]


Epoch 7/10 - Loss: 0.1912 - Accuracy: 94.11%


Epoch 8/10:  35%|███▍      | 131/375 [43:57<1:21:53, 20.14s/it]
